<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EC%A6%9D%EB%B6%84/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8S%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본			   G_inc_so_model_노이2.pt
 산업분류자동화		   G_inc_so_model_노이.pt
'Colab Notebooks'	  'My Drive'
 G_inc_again_so_model.pt   Screenshot_20220401-001411_Chrome.jpg


In [35]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/S_spell_check.csv', sep = ",", encoding = "UTF-8")

## 데이터 전처리

In [36]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차 부분정비 타이어 오일 교환
1,id_0000017,S,95,952,사업장에서 고객 요청으로 자동차세 창업
2,id_0000029,S,96,961,미용실에서 일반인 대상 컷. 파마
3,id_0000034,S,96,969,세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁
4,id_0000037,S,94,949,교회 기독교 계통 종교활동 종교 서비스
...,...,...,...,...,...
100391,id_0999945,S,95,952,사업장에서 고객 요청으로 자동차 경정비
100392,id_0999958,S,94,941,지부에서 회원들의 친목도모 회원관리
100393,id_0999967,S,96,961,미용실 일반인 대상 미용
100394,id_0999979,S,95,951,매장에서 고객의 의뢰에 의해 핸드폰 수리


In [37]:
df['digit_3'].value_counts().sort_index()

941     1612
942      433
949    21580
951     1508
952    12142
953     6911
961    43040
969    13170
Name: digit_3, dtype: int64

## **# 데이터 증분**

In [38]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [39]:
df0=df.loc[(df['digit_3'] == 941)]  #*10
df1=df.loc[(df['digit_3'] == 942)] #*50
df2=df.loc[(df['digit_3'] == 949)]  
df3=df.loc[(df['digit_3'] == 951)]  #*10
df4=df.loc[(df['digit_3'] == 952)]   
df5=df.loc[(df['digit_3'] == 953)]   #*2
df6=df.loc[(df['digit_3'] == 961)]   
df7=df.loc[(df['digit_3'] == 969)]   

In [40]:
df1=df1.reset_index(drop=True)
df2=df2.reset_index(drop=True)
df3=df3.reset_index(drop=True)
df4=df4.reset_index(drop=True)
df5=df5.reset_index(drop=True)
df6=df6.reset_index(drop=True)
df7=df7.reset_index(drop=True)

In [41]:
df_1=pd.concat([df2, df4, df6, df7]) # *1
df_2=pd.concat([df5]) # *2
df_10=pd.concat([df0, df3]) # *10 묶음
df_50=pd.concat([df1]) # *50

In [42]:
df_1=df_1.reset_index(drop=True)
df_2=df_2.reset_index(drop=True)
df_10=df_10.reset_index(drop=True)
df_50=df_50.reset_index(drop=True)

In [43]:
df1.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000804,S,94,942,사무실에서 직원들의 권익 보호 실현 근로조건 유지 개선
1,id_0001348,S,94,942,사무실에서 직원 복지를 위한 노동조합
2,id_0006074,S,94,942,"피고용자들이 조합을 결성하여 회원관리, 전문가 단체"
3,id_0006606,S,94,942,피고용자로 구성된 단체 근로자 근로조건 개선 유지를 위해 구성됨 노조단체
4,id_0033182,S,94,942,"사업장 내에서 직원 복지, 관리 노동조합"


In [44]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df_2['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=['p'+str(df_2['AI_id'][n]), 'p'+str(df_2['AI_id'][n])]
  digit_1_list=[df_2['digit_1'][n], df_2['digit_1'][n]]
  digit_2_list=[df_2['digit_2'][n], df_2['digit_2'][n]]
  digit_3_list=[df_2['digit_3'][n], df_2['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_2_to_insert = pd.DataFrame(data_to_insert)
  df_2= df_2.append(df_2_to_insert, ignore_index=True)
  n=n+1

In [45]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_10['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=9)
  id_list=[str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n]), 'p'+str(df_10['AI_id'][n])]
  digit_1_list=[df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n], df_10['digit_1'][n]]
  digit_2_list=[df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n], df_10['digit_2'][n]]
  digit_3_list=[df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n], df_10['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_10_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_10_to_insert, ignore_index=True)
  n=n+1

df_10=temp

In [46]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

temp = {'AI_id': [], 'digit_1': [], 'digit_2' : [], 'digit_3' : [], 'text' : []}
temp = pd.DataFrame(temp)

for sentence in df_50['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=49)
  id_list=[str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n]), 'p'+str(df_50['AI_id'][n])]
  digit_1_list=[df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n], df_50['digit_1'][n]]
  digit_2_list=[df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n], df_50['digit_2'][n]]
  digit_3_list=[df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n], df_50['digit_3'][n]]

  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df_50_to_insert = pd.DataFrame(data_to_insert)
  temp= temp.append(df_50_to_insert, ignore_index=True)
  n=n+1

df_50=temp

기존 데이터 합치기

In [47]:
df=pd.concat([df_1, df_2, df_10, df_50])

In [48]:
df['digit_3'].value_counts().sort_index()

941.0    16120
942.0    21650
949.0    21580
951.0    15080
952.0    12142
953.0    20733
961.0    43040
969.0    13170
Name: digit_3, dtype: int64

In [49]:
df.head()

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000037,S,94.0,949.0,교회 기독교 계통 종교활동 종교 서비스
1,id_0000040,S,94.0,949.0,교회에서 기독교 계통의 종교활동 종교 서비스
2,id_0000058,S,94.0,949.0,교회 기독교 계통 종교활동 종교 서비스
3,id_0000115,S,94.0,949.0,협회에서 게스트하우스 대상으로 관리 및 진흥
4,id_0000135,S,94.0,949.0,교회에서 신자들 대상으로 기독교 계통 종교활동


In [50]:
df.to_csv('df_S_inc.csv', index=False, encoding='utf-8-sig')